In [ ]:
import networkx as nx
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import matplotlib.font_manager
from scipy.stats import pearsonr
from scipy.stats import linregress
from matplotlib import pyplot as plt
import matplotlib as mpl
from pycirclize import Circos
matplotlib.font_manager.fontManager.addfont('/h/tianyi/TS_datasets_reversion/Cell_Press_plot/Arial.ttf')
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
mpl.rcParams['font.size'] = 8 
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.titlecolor'] = 'black'
plt.rcParams['legend.labelcolor'] = 'black'
plt.rcParams['axes.linewidth'] = 0.5

In [ ]:
os.chdir('/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/data_statistic/share_enrich_pathway')
enrich_p_value=pd.read_csv('gsea_reactome_enrichment_tissue_primary_all_v2.txt',sep='\t')
primary_subnet_count=pd.read_csv('top20_pathway_share.txt',sep='\t')

library(ggpubr)
library("ggsci")
library(ggplot2)
library(ggsignif)
library(ggpubr)
library(ComplexHeatmap)
library(circlize)


########################specific pathway#########################
setwd('G:\\课题\\tissue_specificity_manuscript\\素材\\share_enrich_pathway')
enrich_p_value=data.frame(read.table('gsea_reactome_enrichment_tissue_primary_all_v2.txt',header = T,sep='\t', row.names = 1))
primary_subnet_count=data.frame(read.table('top20_pathway_share.txt',sep='\t',header = T, row.names = 1))
enrich_p_value_share=enrich_p_value[which(rownames(enrich_p_value) %in% rownames(primary_subnet_count)),]
enrich_p_value_log10=-log10(as.matrix(enrich_p_value_share))

tissue_anno1=gsub('Primary_','',colnames(enrich_p_value_log10))
tissue_anno2=lapply(tissue_anno1, 
                    function(x) {
                      str1_pro=strsplit(x, "_")[[1]]
                      str2_pro=head(str1_pro,-1)
                      paste(str2_pro, collapse = "_")
                    }
)
color_df=data.frame(read.table('tissue_color.txt',sep='\t',header = 1,quote='',fill=T,comment.char =''))
tissue_all=color_df$tissue
color_all=color_df$tissue_cor_v2
names(color_all)=tissue_all

tissue_color_list=list(Tissue=color_all)
tissue_df=data.frame(unlist(tissue_anno2))
colnames(tissue_df)='Tissue'
tissue_anno = HeatmapAnnotation(df =tissue_df, col = tissue_color_list)
plot(tissue_anno)
gene_all=lapply(tissue_anno1, 
                function(x) {
                  tail(unlist(strsplit(x, "_")),n=1)
                })

gene_all_sort=unlist(gene_all)
get_first_letter <- function(string) {
  substr(string, 1, 1)
}
gene_all_sort_pro <- sort(gene_all_sort)
gene_all_unique=unique(unlist(gene_all_sort_pro))
color_all_gene= colorRampPalette(c('#E3475A','#FFFFFF','#626EB3'))(n = length(gene_all_unique))
names(color_all_gene)=gene_all_unique

#############设置颜色#######
anno_df = data.frame(Tissue = tissue_df$Tissue,
                     Gene = unlist(gene_all))

col_fun= colorRamp2(c(0,-log10(0.05),40),c('#80AACA','#FFFFFF','#EE7B6C'))
colnames(enrich_p_value_log10)=lapply(tissue_anno1, 
                                      function(x) {
                                        tail(unlist(strsplit(x, "_")),n=1)
                                      })

tissue_gene_annot = HeatmapAnnotation(df = anno_df,
                                      col = list(Tissue = color_all,
                                                 Gene =color_all_gene
                                      ),show_legend =FALSE
)


pdf('share_pathway_pvalue_nolegend.pdf',width=8, height=(11/3))
pushViewport(viewport(gp = gpar(fontfamily = "sans")))
Heatmap_p=Heatmap(enrich_p_value_log10,name = "-log10(p value)",col=col_fun,show_column_dend = FALSE, show_row_dend = FALSE
                  ,heatmap_legend_param = list(legend_direction = "horizontal",title_gp  = gpar(fontsize = 8), labels_gp = gpar(fontsize = 8))
                  ,row_names_gp = gpar(fontsize = 8),column_names_gp  = gpar(fontsize = 8),  top_annotation  = tissue_gene_annot,show_column_names=F
)
draw(Heatmap_p, newpage = FALSE)
popViewport()
dev.off()


In [ ]:
## share pathway subnetwork number
top20_pathway_num=pd.read_csv('/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/data_statistic/share_enrich_pathway/top20_pathway_share_v2.txt',sep='\t')

fig, axes = plt.subplots(1,1,figsize=(1.5,4*(2/3)))
plt.rcParams['axes.linewidth'] = 0.5
sns.barplot(data=top20_pathway_num,x='enrich_counts',y='Pathway_name',color='#CC3C4C',ax=axes)
axes.spines['top'].set_visible(False)
axes.spines['right'].set_visible(False)
axes.set_xlabel('Number of significantly enriched subnetworks',fontsize=8, fontname='Arial')
axes.set_ylabel('',fontsize=8, fontname='Arial')
axes.tick_params(axis='x', labelsize=8, width=0.5)
axes.tick_params(axis='y', labelsize=8, width=0.5)
plt.savefig('/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/data_statistic/share_enrich_pathway/share_enrich_pathway_subnetwork_number_v4.pdf',dpi=300,bbox_inches='tight')


In [ ]:
## specific pathway
library(clusterProfiler)
library(ggplot2)
setwd('C:\\Users\\tianyi\\Desktop\\coding\\subnetwork_number_heatmap')
specific_pathway_p_GR_all=data.frame(read.table('specific_pathway_p_GR_all.txt',sep='\t',header=TRUE))
tissue_color=data.frame(read.table('G:\\课题\\tissue_specificity_manuscript\\素材\\specific_enrich_pathway\\tissue_color.txt',sep='\t',header=T))
tissue_color=tissue_color[,c('tissue','tissue_name')]
specific_pathway_p_GR_all=merge(specific_pathway_p_GR_all,tissue_color,by='tissue')
specific_pathway_p_GR_all$pathway_id_plot <- factor(specific_pathway_p_GR_all$pathway_id_pro, levels = unique(specific_pathway_p_GR_all$pathway_id_pro))
specific_pathway_p_GR_all$Tissue_gene=paste(specific_pathway_p_GR_all$tissue_name,specific_pathway_p_GR_all$seed_gene,sep=' ')
specific_pathway_p_GR_all_sort=specific_pathway_p_GR_all[order(specific_pathway_p_GR_all$Tissue_gene),]

pdf('G:\\课题\\tissue_specificity_manuscript\\素材\\specific_enrich_pathway\\specific_pathway_plot_v2.pdf',height = 4, width = 8/2)
ggplot(specific_pathway_p_GR_all_sort, aes(Tissue_gene,pathway_id_plot ), showCategory=1) +
  geom_point(aes(color=FDR_BH , size=GeneRatio))+theme(axis.text.x = element_text(angle = 90, hjust = 1, size = 8),
                                                       axis.line.y = element_line(size = (0.5/1.07)*0.5),
                                                       axis.line.x = element_line(size = (0.5/1.07)*0.5) ,
                                                       axis.ticks.y = element_line(size = (0.5/1.07)*0.5) ,
                                                       axis.ticks.x = element_line(size = (0.5/1.07)*0.5) ,
                                                       axis.text.y = element_text(size = 8)  ,
                                                       axis.title.x = element_blank()  ,axis.title.y = element_blank(),
                                                       panel.border = element_rect(color = "black", fill = NA, size = 0.5),
                                                       panel.background = element_blank())+ 
  scale_color_gradient(low = "#E3475A", high = "#626EB3",limits = c(0, 0.05))+scale_size_continuous(range = c(0.5, 3))+
  theme(legend.position = "none") +theme(panel.grid.minor = element_blank(), panel.grid.major = element_blank())
dev.off()

pdf('C:\\Users\\tianyi\\Desktop\\coding\\subnetwork_number_heatmap\\specific_pathway_plot_legend.pdf',height = 4, width = 8)
ggplot(specific_pathway_p_GR_all_sort, aes(Tissue_gene,pathway_id_plot ), showCategory=1) +
  geom_point(aes(color=FDR_BH , size=GeneRatio))+theme(axis.text.x = element_text(angle = 90, hjust = 1, size = 8),
                                                       axis.text.y = element_text(size = 8)  ,axis.title.x = element_blank()  ,axis.title.y = element_blank(),
                                                       panel.border = element_rect(color = "black", fill = NA, size = 0.5),
                                                       panel.background = element_blank())+ 
  scale_color_gradient(low = "#E3475A", high = "#626EB3",limits = c(0, 0.05))+scale_size_continuous(range = c(0.5, 3))+
  theme(panel.grid.minor = element_blank(), panel.grid.major = element_blank())+theme(legend.position = "bottom")
dev.off()


In [ ]:
## gene number in subnetwork
library(dplyr)
library(paletteer)
library(ggplot2)
library(ggpubr)
library("ggsci")
library(stringr)
setwd('C:\\Users\\tianyi\\Desktop\\coding\\subnetwork_number_heatmap\\r_barplot')
subnetwork_number=data.frame(read.table('primary_subnetwork_gene_count_all_subnetwork_level_barplot_v2.txt',sep='\t',header=1))
subnetwork_number$subnetwork_id=str_c(subnetwork_number$tissue,'-',subnetwork_number$seed)
subnetwork_number_sort=subnetwork_number[order(subnetwork_number$gene_number,decreasing = T),]

subnetwork_number_sort_member_sum=subnetwork_number_sort %>%
  group_by(subnetwork_id) %>%
  summarise(total_gene_number = sum(gene_number))
subnetwork_number_sort_member_sum=subnetwork_number_sort_member_sum[order(subnetwork_number_sort_member_sum$total_gene_number,decreasing = F),]
subnetwork_number_sort$subnetwork_id=factor(subnetwork_number_sort$subnetwork_id, levels = unique(subnetwork_number_sort_member_sum$subnetwork_id))
subnetwork_number_sort$classcification=factor(subnetwork_number_sort$classcification, levels =c("Tissue-specific genetic mutations & expressed genes & driver genes",
                                                                                                "Tissue-specific genetic mutations & expressed genes",
                                                                                                "Tissue-specific genetic mutations & driver genes",
                                                                                                "Tissue-specific expressed genes & driver genes",
                                                                                                "Only tissue-specific genetic mutations",
                                                                                                "Only tissue-specific expressed genes",
                                                                                                "Only driver genes",
                                                                                                "Subnetwork member genes"
))



group_colors= c("Subnetwork member genes" = "#CC3C4C", 
                "Only driver genes" = "#3682B3",
                "Only tissue-specific expressed genes" = "#E5C87A",
                "Tissue-specific genetic mutations & expressed genes" = "#CDE3EB",
                "Tissue-specific genetic mutations & expressed genes & driver genes"= "#8F91C2",
                "Only tissue-specific genetic mutations"= "#ADCFA1",
                "Tissue-specific genetic mutations & driver genes"= "#7CB964",
                "Tissue-specific expressed genes & driver genes"= "#E597B5")  # 请根据实际情况设置颜色

pdf('G:/课题/tissue_specificity_manuscript/素材/subnetwork_member_number_barplot/sunbnetwork_number_stack_barplot_nolegend_position_v4.pdf',width=8, height=(11/3)*(2/3))
ggplot(data = subnetwork_number_sort,aes(x=gene_number,y=subnetwork_id,group=classcification ))+
  geom_bar(stat = "identity" , position="stack", width =1 ,aes(fill=classcification) ) +  theme_minimal()+
  scale_fill_manual(values = group_colors)+ ##用于手动设置离散型填充颜色的函数
  theme(legend.position = "none",    axis.ticks.y = element_blank(),  axis.text.y = element_blank(),
        axis.line = element_line(size = 0.703125 ),
        panel.grid=element_blank(),
        axis.line.y = element_line(size = (0.5/1.07)*0.5),
        axis.line.x = element_line(size = (0.5/1.07)*0.5) ,
        axis.ticks.x = element_line(size = (0.5/1.07)*0.5) ,
        axis.text.x = element_text(size = 8, family = "sans", 
                                   color = "black")) + # 去除 y 轴刻度线
  xlab('Subnetwork genes number')+ylab('Subnetwork')+
  font("xlab",size = 8, family = "sans", color = "black")+
  font("ylab",size = 8, family = "sans", color = "black")+
  scale_y_discrete(expand = c(0, 0)) +  # 确保 y 轴从原点开始
  scale_x_continuous(expand = c(0, 0), limits = c(0, max(subnetwork_number_sort$gene_number) + 15))
dev.off()

subnetwork_number_sort_rank=data.frame(unique(subnetwork_number_sort[['subnetwork_id']]))
subnetwork_number_sort_rank['rank']=c(1:dim(subnetwork_number_sort_rank)[1])
subnetwork_number_sort_rank['posi']=seq(280.52,361.692, 280.894-280.52)
colnames(subnetwork_number_sort_rank)=c('subnetwork_id','rank','posi')
subnetwork_number_sort_rank[which(subnetwork_number_sort_rank['subnetwork_id']=='Lung-KRAS'),]